In [1]:
from tensorflow.examples.tutorials.mnist import input_data
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
import tensorflow as tf
import numpy as np
import random

# 将 numpy 数组中的图片和标签顺序打乱
def shuffer_images_and_labels(images, labels):
    shuffle_indices = np.random.permutation(np.arange(len(images)))
    shuffled_images = images[shuffle_indices]
    shuffled_labels = labels[shuffle_indices]
    return shuffled_images, shuffled_labels

# 将label从长度10的one hot向量转换为0~9的数字
# 例：get_label(total_labels[0]) 获取到total_labels中第一个标签对应的数字
def get_label(label):
    return np.argmax(label)

# images：训练集的feature部分
# labels：训练集的label部分
# batch_size： 每次训练的batch大小
# epoch_num： 训练的epochs数
# shuffle： 是否打乱数据
# 使用示例：
#   for (batchImages, batchLabels) in batch_iter(images_train, labels_train, batch_size, epoch_num, shuffle=True):
#       sess.run(feed_dict={inputLayer: batchImages, outputLabel: batchLabels})
def batch_iter(images,labels, batch_size, epoch_num, shuffle=True):
    data_size = len(images)
    num_batches_per_epoch = int(data_size / batch_size)  # 样本数/batch块大小,多出来的“尾数”，不要了
    for epoch in range(epoch_num):
        # Shuffle the data at each epoch
        if shuffle:
            shuffle_indices = np.random.permutation(np.arange(data_size))
            shuffled_data_feature = images[shuffle_indices]
            shuffled_data_label   = labels[shuffle_indices]
        else:
            shuffled_data_feature = images
            shuffled_data_label = labels
        for batch_num in range(num_batches_per_epoch):   # batch_num取值0到num_batches_per_epoch-1
            start_index = batch_num * batch_size
            end_index = min((batch_num + 1) * batch_size, data_size)

            yield (shuffled_data_feature[start_index:end_index] , shuffled_data_label[start_index:end_index])


# 读取数据集
mnist = input_data.read_data_sets('MNIST_data/', one_hot=True)

total_images = mnist.train.images
total_labels = mnist.train.labels
total_images, total_labels = shuffer_images_and_labels(total_images, total_labels)

# 简单划分前50000个为训练集，后5000个为测试集
origin_images_train = total_images[:50000]
origin_labels_train = total_labels[:50000]
origin_images_test = total_images[50000:]
origin_labels_test = total_labels[50000:]

# 构建和训练模型
def train_and_test1(images_train, labels_train, images_test, labels_test, images_validation, labels_validation):
    x = tf.placeholder(tf.float32,[None,784],name = "X")
    y = tf.placeholder(tf.float32,[None,10],name = "Y")
   
    #隐藏层神经元数量
    H1_NN = 256#第1隐藏层神经元为256个
    H2_NN = 64#第2隐藏层神经元为64个
    #输入层 - 第1隐藏层参数和偏置顶
    W1 = tf.Variable(tf.truncated_normal([784,H1_NN],stddev = 0.1))
    b1 = tf.Variable(tf.zeros([H1_NN]))
    #第1隐藏层- 第2隐藏层参数和偏置顶
    W2 = tf.Variable(tf.truncated_normal([H1_NN,H2_NN],stddev = 0.1))
    b2 = tf.Variable(tf.zeros([H2_NN]))
    #第2隐藏层- 输出层
    W3 = tf.Variable(tf.truncated_normal([H2_NN,10],stddev = 0.1))
    b3 = tf.Variable(tf.zeros([10]))
    Y1 = tf.nn.relu(tf.matmul(x,W1)+b1)
    Y2 = tf.nn.relu(tf.matmul(Y1,W2)+b2)
    forward = tf.matmul(Y2,W3)+b3
    pred = tf.nn.softmax(forward)
    #交叉熵
    loss_function = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=forward,labels=y))
    train_epochs = 40#训练轮数
    batch_size = 60#单次训练样本数
    total_batch = int(len(images_train)/batch_size)#一轮训练有多少批次
    #splay_step = 1#显示粒度
    learning_rate = 0.003#学习率
    #优化器
    optimizer = tf.train.AdamOptimizer(learning_rate).minimize(loss_function)
    correct_prediction = tf.equal(tf.argmax(y,1),tf.argmax(pred,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))
    #建立会话
    sess = tf.Session()
    init = tf.global_variables_initializer()
    sess.run(init)
    count = 1
    for (batchImages, batchLabels) in batch_iter(images_train, labels_train, batch_size, train_epochs, shuffle=True):
        sess.run(optimizer,feed_dict={x: batchImages, y: batchLabels})
        # accu_test = sess.run(accuracy,feed_dict={x:images_test,y:labels_test})
        
        if count%total_batch== 0:
            accu_test = sess.run(accuracy,feed_dict={x:images_test,y:labels_test})
            print("Accuracy:",accu_test)  
        count += 1
    print("Final Accuracy:",sess.run(accuracy,feed_dict={x:images_validation,y:labels_validation}))
    sess.close()
    print("Train finied!")
# 划分数据集并调用train_and_test测试和验证

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [2]:
# 使用简单划分的训练集和测试集训练，并使用测试集评估模型
from time import time
start_time=time()
train_and_test1(origin_images_train, origin_labels_train, origin_images_test, origin_labels_test, origin_images_test, origin_labels_test)
print("during_time:",time()-start_time)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

Accuracy: 0.9612
Accuracy: 0.9578
Accuracy: 0.9708
Accuracy: 0.9716
Accuracy: 0.9736
Accuracy: 0.974
Accuracy: 0.9716
Accuracy: 0.9768
Accuracy: 0.9726
Accuracy: 0.9722
Accuracy: 0.9722
Accuracy: 0.971
Accuracy: 0.9762
Accuracy: 0.9742
Accuracy: 0.9748
Accuracy: 0.9746
Accuracy: 0.972
Accuracy: 0.972
Accuracy: 0.9726
Accuracy: 0.9728
Accuracy: 0.9698
Accuracy: 0.9756
Accuracy: 0.9756
Accuracy: 0.9768
Accuracy: 0.9764
Accuracy: 0.9754
Accuracy: 0.9686
Accuracy: 0.9736
Accuracy: 0.977
Accuracy: 0.9752
Accuracy: 0.976
Accuracy: 0.9722
Accuracy: 0.9752
Accuracy: 0.9746
Accuracy: 0.9784
Accuracy: 0.975
Accuracy: 0.9746
Accuracy: 0.9772
Accuracy: 0.978
Accuracy: 0.9746
Final Accuracy: 0.9746
Train finied!
during_time: 165.0057349205017


In [3]:
def train_and_test2(images_train, labels_train, images_test, labels_test, images_validation, labels_validation):
    x = tf.placeholder(tf.float32,[None,784],name = "X")
    y = tf.placeholder(tf.float32,[None,10],name = "Y")
    #1层隐藏层
    ##隐藏层神经元数量
    H1_NN = 256
    W1 = tf.Variable(tf.random_normal([784,H1_NN]))
    b1 = tf.Variable(tf.zeros([H1_NN]))
    #激活函数ReLU
    Y1 = tf.nn.relu(tf.matmul(x,W1)+b1)
    #构建输出层
    W2 = tf.Variable(tf.random_normal([H1_NN,10]))
    b2 = tf.Variable(tf.zeros([10]))
    #前向计算
    forward = tf.matmul(Y1,W2) + b2
    pred = tf.nn.softmax(forward)
    #交叉熵
    loss_function = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=forward,labels=y))
    train_epochs = 50#训练轮数
    batch_size = 200#单次训练样本数
    total_batch = int(len(images_train)/batch_size)#一轮训练有多少批次
    learning_rate = 0.01#学习率
    #优化器
    optimizer = tf.train.AdamOptimizer(learning_rate).minimize(loss_function)
    correct_prediction = tf.equal(tf.argmax(y,1),tf.argmax(pred,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))
    #建立会话
    sess = tf.Session()
    init = tf.global_variables_initializer()
    sess.run(init)
    count = 0
    for (batchImages, batchLabels) in batch_iter(images_train, labels_train, batch_size, train_epochs, shuffle=True):
        sess.run(optimizer,feed_dict={x: batchImages, y: batchLabels})
        if count%total_batch== 0:
            accu_test = sess.run(accuracy,feed_dict={x:images_test,y:labels_test})
            print("Accuracy:",accu_test)  
        count += 1
    #accu_validation = sess.run(accuracy,feed_dict={x:images_validation,y:labels_validation})
    #print("Accuracy:",accu_validation)
    print("Accuracy:",accu_test)
    sess.close()
  #  return accu_validation.item()
    return accu_test.item()
# 划分数据集并调用train_and_test测试和验证

In [4]:
# 使用简单划分的训练集和测试集训练，并使用测试集评估模型
from time import time
start_time=time()
train_and_test2(origin_images_train, origin_labels_train, origin_images_test, origin_labels_test, origin_images_test, origin_labels_test)
print("during_time:",time()-start_time)

Accuracy: 0.171
Accuracy: 0.9108
Accuracy: 0.9254
Accuracy: 0.944
Accuracy: 0.9444
Accuracy: 0.9492
Accuracy: 0.9562
Accuracy: 0.9572
Accuracy: 0.956
Accuracy: 0.958
Accuracy: 0.959
Accuracy: 0.9556
Accuracy: 0.9608
Accuracy: 0.9636
Accuracy: 0.9622
Accuracy: 0.9638
Accuracy: 0.963
Accuracy: 0.9646
Accuracy: 0.9622
Accuracy: 0.969
Accuracy: 0.9654
Accuracy: 0.9674
Accuracy: 0.9646
Accuracy: 0.9696
Accuracy: 0.9654
Accuracy: 0.9664
Accuracy: 0.9668
Accuracy: 0.9644
Accuracy: 0.9664
Accuracy: 0.9672
Accuracy: 0.9656
Accuracy: 0.9698
Accuracy: 0.9666
Accuracy: 0.9684
Accuracy: 0.9676
Accuracy: 0.9704
Accuracy: 0.9696
Accuracy: 0.9708
Accuracy: 0.9672
Accuracy: 0.9686
Accuracy: 0.9718
Accuracy: 0.9678
Accuracy: 0.9704
Accuracy: 0.9684
Accuracy: 0.971
Accuracy: 0.9732
Accuracy: 0.9684
Accuracy: 0.9688
Accuracy: 0.9704
Accuracy: 0.9714
Accuracy: 0.9714
during_time: 79.71956300735474


In [5]:
def hold_out(images, labels, train_percentage):
    images_train,images_test,\
    labels_train,labels_test=train_test_split(images, labels,test_size=train_percentage,random_state=0)
    train_and_test2(images_train, labels_train, images_test, labels_test,images_test,labels_test)

In [6]:
print(total_images.shape[1])
print(origin_images_test.shape)

784
(5000, 784)


In [7]:
# 调用函数用留出法和k折交叉验证法评估模型
hold_out(total_images, total_labels, 0.2)


Accuracy: 0.08209091
Accuracy: 0.90727276
Accuracy: 0.92645454
Accuracy: 0.935
Accuracy: 0.943
Accuracy: 0.9463636
Accuracy: 0.95045453
Accuracy: 0.9482727
Accuracy: 0.9522727
Accuracy: 0.9559091
Accuracy: 0.9541818
Accuracy: 0.953
Accuracy: 0.9515455
Accuracy: 0.95754546
Accuracy: 0.9589091
Accuracy: 0.96354544
Accuracy: 0.95363635
Accuracy: 0.962
Accuracy: 0.9656364
Accuracy: 0.9602727
Accuracy: 0.96427274
Accuracy: 0.9681818
Accuracy: 0.96927273
Accuracy: 0.96736366
Accuracy: 0.96763635
Accuracy: 0.9645454
Accuracy: 0.96163636
Accuracy: 0.96754545
Accuracy: 0.9662727
Accuracy: 0.967
Accuracy: 0.9637273
Accuracy: 0.9637273
Accuracy: 0.9694545
Accuracy: 0.96645457
Accuracy: 0.9696364
Accuracy: 0.9671818
Accuracy: 0.96745455
Accuracy: 0.96909094
Accuracy: 0.9681818
Accuracy: 0.9699091
Accuracy: 0.97145456
Accuracy: 0.97227275
Accuracy: 0.96936363
Accuracy: 0.9696364
Accuracy: 0.96663636
Accuracy: 0.97136366
Accuracy: 0.96845454
Accuracy: 0.97081816
Accuracy: 0.96827275
Accuracy: 0.9717

In [8]:
hold_out(total_images, total_labels, 0.4)

Accuracy: 0.12127273
Accuracy: 0.8989546
Accuracy: 0.91804546
Accuracy: 0.9334091
Accuracy: 0.9330909
Accuracy: 0.93872726
Accuracy: 0.9425909
Accuracy: 0.9439545
Accuracy: 0.946
Accuracy: 0.948
Accuracy: 0.9447727
Accuracy: 0.9465455
Accuracy: 0.95104545
Accuracy: 0.9534091
Accuracy: 0.95345455
Accuracy: 0.9493182
Accuracy: 0.9538182
Accuracy: 0.9581364
Accuracy: 0.9559091
Accuracy: 0.9579545
Accuracy: 0.9565909
Accuracy: 0.9561818
Accuracy: 0.95295453
Accuracy: 0.9575
Accuracy: 0.95668185
Accuracy: 0.9604091
Accuracy: 0.9615909
Accuracy: 0.9606364
Accuracy: 0.9612727
Accuracy: 0.9625
Accuracy: 0.9628182
Accuracy: 0.9612273
Accuracy: 0.9635909
Accuracy: 0.95931816
Accuracy: 0.96363634
Accuracy: 0.95759094
Accuracy: 0.9614546
Accuracy: 0.96195453
Accuracy: 0.96254545
Accuracy: 0.9645
Accuracy: 0.96522725
Accuracy: 0.9645909
Accuracy: 0.9614546
Accuracy: 0.96509093
Accuracy: 0.9662273
Accuracy: 0.9655
Accuracy: 0.9671818
Accuracy: 0.9627727
Accuracy: 0.96686363
Accuracy: 0.9653636
Accur

In [9]:
hold_out(total_images, total_labels, 0.6)

Accuracy: 0.099454544
Accuracy: 0.8800909
Accuracy: 0.909
Accuracy: 0.9147879
Accuracy: 0.92045456
Accuracy: 0.92857575
Accuracy: 0.9290606
Accuracy: 0.9321515
Accuracy: 0.93133336
Accuracy: 0.9369091
Accuracy: 0.93666667
Accuracy: 0.93742424
Accuracy: 0.94042426
Accuracy: 0.9412121
Accuracy: 0.94069695
Accuracy: 0.9394242
Accuracy: 0.9364849
Accuracy: 0.94266665
Accuracy: 0.94618183
Accuracy: 0.94690907
Accuracy: 0.9456364
Accuracy: 0.9467273
Accuracy: 0.9489394
Accuracy: 0.9486667
Accuracy: 0.94536364
Accuracy: 0.94912124
Accuracy: 0.9470303
Accuracy: 0.94954544
Accuracy: 0.9471818
Accuracy: 0.9460606
Accuracy: 0.9556364
Accuracy: 0.95160604
Accuracy: 0.9522727
Accuracy: 0.95363635
Accuracy: 0.9518485
Accuracy: 0.9529697
Accuracy: 0.95427275
Accuracy: 0.9558182
Accuracy: 0.95869696
Accuracy: 0.95839393
Accuracy: 0.9569091
Accuracy: 0.95736367
Accuracy: 0.95721215
Accuracy: 0.95715153
Accuracy: 0.960303
Accuracy: 0.9560303
Accuracy: 0.9588182
Accuracy: 0.9551212
Accuracy: 0.9580606
Ac

In [10]:
hold_out(total_images, total_labels, 0.8)

Accuracy: 0.16654545
Accuracy: 0.85643184
Accuracy: 0.87520456
Accuracy: 0.8882046
Accuracy: 0.8971591
Accuracy: 0.90338635
Accuracy: 0.909
Accuracy: 0.90820456
Accuracy: 0.9112273
Accuracy: 0.9135454
Accuracy: 0.9138409
Accuracy: 0.9178409
Accuracy: 0.91870457
Accuracy: 0.91804546
Accuracy: 0.91984093
Accuracy: 0.9217727
Accuracy: 0.9191591
Accuracy: 0.9225
Accuracy: 0.9236591
Accuracy: 0.9226818
Accuracy: 0.92397726
Accuracy: 0.9245682
Accuracy: 0.92175
Accuracy: 0.9250682
Accuracy: 0.9229091
Accuracy: 0.9311818
Accuracy: 0.9267955
Accuracy: 0.9309545
Accuracy: 0.9285227
Accuracy: 0.9276364
Accuracy: 0.9270909
Accuracy: 0.9365
Accuracy: 0.9366136
Accuracy: 0.9345
Accuracy: 0.9363409
Accuracy: 0.9359091
Accuracy: 0.93384093
Accuracy: 0.9353409
Accuracy: 0.93529546
Accuracy: 0.9359773
Accuracy: 0.92979544
Accuracy: 0.93131816
Accuracy: 0.9384773
Accuracy: 0.93784094
Accuracy: 0.9334091
Accuracy: 0.9394091
Accuracy: 0.93825
Accuracy: 0.9366136
Accuracy: 0.9423409
Accuracy: 0.9407727
Acc

In [11]:
hold_out(total_images, total_labels, 0.9)

Accuracy: 0.15470707
Accuracy: 0.81713134
Accuracy: 0.8583232
Accuracy: 0.8672323
Accuracy: 0.87218183
Accuracy: 0.8753737
Accuracy: 0.88456565
Accuracy: 0.8810303
Accuracy: 0.88638383
Accuracy: 0.89226264
Accuracy: 0.8919192
Accuracy: 0.88953537
Accuracy: 0.891697
Accuracy: 0.89270705
Accuracy: 0.8931313
Accuracy: 0.89246464
Accuracy: 0.8942424
Accuracy: 0.8952121
Accuracy: 0.8951111
Accuracy: 0.8951111
Accuracy: 0.89507073
Accuracy: 0.8951313
Accuracy: 0.8951515
Accuracy: 0.8951313
Accuracy: 0.8951717
Accuracy: 0.8951515
Accuracy: 0.8951515
Accuracy: 0.8952323
Accuracy: 0.8952323
Accuracy: 0.8952525
Accuracy: 0.8952323
Accuracy: 0.8951919
Accuracy: 0.8952121
Accuracy: 0.8952323
Accuracy: 0.89533335
Accuracy: 0.89537376
Accuracy: 0.89537376
Accuracy: 0.8954545
Accuracy: 0.8954545
Accuracy: 0.8954545
Accuracy: 0.8954545
Accuracy: 0.8954343
Accuracy: 0.89539397
Accuracy: 0.89547473
Accuracy: 0.89547473
Accuracy: 0.89547473
Accuracy: 0.8954545
Accuracy: 0.89549494
Accuracy: 0.8954545
Acc

In [12]:
def cross_validation(images, labels, k):
    count1 = 0.0
    kf = KFold(n_splits =k,shuffle= True)
    i = 1
    for train_index, test_index in kf.split(images):
        images_train,images_test = images[train_index],images[test_index]
        labels_train,labels_test = labels[train_index],labels[test_index]
        print("Train:",i)
        i +=1
        temp=train_and_test2(images_train, labels_train, images_test, labels_test, images_test, labels_test)
        count1 = count1 + temp
    average = count1/k
    print("Average:",average)        


In [13]:
cross_validation(origin_images_train, origin_labels_train, 5)

Train: 1
Accuracy: 0.1009
Accuracy: 0.9021
Accuracy: 0.9238
Accuracy: 0.9351
Accuracy: 0.9419
Accuracy: 0.9486
Accuracy: 0.9496
Accuracy: 0.9507
Accuracy: 0.9507
Accuracy: 0.9553
Accuracy: 0.9554
Accuracy: 0.9558
Accuracy: 0.956
Accuracy: 0.9589
Accuracy: 0.9566
Accuracy: 0.9583
Accuracy: 0.9611
Accuracy: 0.9607
Accuracy: 0.9602
Accuracy: 0.9642
Accuracy: 0.9643
Accuracy: 0.9602
Accuracy: 0.9635
Accuracy: 0.9624
Accuracy: 0.962
Accuracy: 0.9658
Accuracy: 0.9635
Accuracy: 0.9653
Accuracy: 0.9649
Accuracy: 0.9617
Accuracy: 0.9654
Accuracy: 0.9664
Accuracy: 0.9662
Accuracy: 0.9694
Accuracy: 0.9679
Accuracy: 0.97
Accuracy: 0.9673
Accuracy: 0.96
Accuracy: 0.9653
Accuracy: 0.9676
Accuracy: 0.9688
Accuracy: 0.9699
Accuracy: 0.9694
Accuracy: 0.9672
Accuracy: 0.9696
Accuracy: 0.9679
Accuracy: 0.971
Accuracy: 0.9679
Accuracy: 0.968
Accuracy: 0.9695
Accuracy: 0.9695
Train: 2
Accuracy: 0.1135
Accuracy: 0.9088
Accuracy: 0.9278
Accuracy: 0.9357
Accuracy: 0.9381
Accuracy: 0.9411
Accuracy: 0.9479
Accu

In [14]:
cross_validation(origin_images_train, origin_labels_train, 10)

Train: 1
Accuracy: 0.0962
Accuracy: 0.8984
Accuracy: 0.9222
Accuracy: 0.9364
Accuracy: 0.9392
Accuracy: 0.945
Accuracy: 0.9448
Accuracy: 0.9486
Accuracy: 0.9476
Accuracy: 0.9558
Accuracy: 0.958
Accuracy: 0.9556
Accuracy: 0.9596
Accuracy: 0.9606
Accuracy: 0.9568
Accuracy: 0.9564
Accuracy: 0.9634
Accuracy: 0.9598
Accuracy: 0.9636
Accuracy: 0.9622
Accuracy: 0.9626
Accuracy: 0.9606
Accuracy: 0.9596
Accuracy: 0.9638
Accuracy: 0.9644
Accuracy: 0.9648
Accuracy: 0.9608
Accuracy: 0.9624
Accuracy: 0.966
Accuracy: 0.9664
Accuracy: 0.9698
Accuracy: 0.9696
Accuracy: 0.9656
Accuracy: 0.968
Accuracy: 0.9686
Accuracy: 0.965
Accuracy: 0.965
Accuracy: 0.9674
Accuracy: 0.9708
Accuracy: 0.9704
Accuracy: 0.969
Accuracy: 0.9666
Accuracy: 0.9698
Accuracy: 0.9668
Accuracy: 0.969
Accuracy: 0.97
Accuracy: 0.9686
Accuracy: 0.9702
Accuracy: 0.9666
Accuracy: 0.9686
Accuracy: 0.9686
Train: 2
Accuracy: 0.1574
Accuracy: 0.9112
Accuracy: 0.9256
Accuracy: 0.9374
Accuracy: 0.947
Accuracy: 0.9478
Accuracy: 0.954
Accuracy

Accuracy: 0.9666
Accuracy: 0.9662
Accuracy: 0.9684
Accuracy: 0.962
Accuracy: 0.9664
Accuracy: 0.9672
Accuracy: 0.963
Accuracy: 0.9612
Accuracy: 0.9664
Accuracy: 0.966
Accuracy: 0.9652
Accuracy: 0.9618
Accuracy: 0.9636
Accuracy: 0.9658
Accuracy: 0.9652
Accuracy: 0.9674
Accuracy: 0.9696
Accuracy: 0.9676
Accuracy: 0.9698
Accuracy: 0.9672
Accuracy: 0.9674
Accuracy: 0.9662
Accuracy: 0.968
Accuracy: 0.969
Accuracy: 0.9712
Accuracy: 0.9704
Accuracy: 0.9704
Average: 0.9704999923706055


In [15]:
from time import time
start_time=time()
cross_validation(origin_images_train, origin_labels_train, 15)
print("during_time:",time()-start_time)

Train: 1
Accuracy: 0.18476304
Accuracy: 0.9085183
Accuracy: 0.925015
Accuracy: 0.9442112
Accuracy: 0.9466107
Accuracy: 0.9517097
Accuracy: 0.9490102
Accuracy: 0.95080984
Accuracy: 0.9514097
Accuracy: 0.9529094
Accuracy: 0.9568086
Accuracy: 0.9577085
Accuracy: 0.9589082
Accuracy: 0.9577085
Accuracy: 0.96340734
Accuracy: 0.9493101
Accuracy: 0.9628074
Accuracy: 0.964907
Accuracy: 0.9628074
Accuracy: 0.97210556
Accuracy: 0.9679064
Accuracy: 0.9604079
Accuracy: 0.9703059
Accuracy: 0.9703059
Accuracy: 0.9670066
Accuracy: 0.96730655
Accuracy: 0.9643071
Accuracy: 0.9661068
Accuracy: 0.9664067
Accuracy: 0.97060585
Accuracy: 0.9640072
Accuracy: 0.9670066
Accuracy: 0.9712058
Accuracy: 0.96670663
Accuracy: 0.96970606
Accuracy: 0.9670066
Accuracy: 0.97210556
Accuracy: 0.9691062
Accuracy: 0.9664067
Accuracy: 0.97090584
Accuracy: 0.96670663
Accuracy: 0.97180563
Accuracy: 0.97480506
Accuracy: 0.97210556
Accuracy: 0.9766047
Accuracy: 0.9736053
Accuracy: 0.9715057
Accuracy: 0.9739052
Accuracy: 0.9694061

Accuracy: 0.9687969
Accuracy: 0.969997
Accuracy: 0.969697
Accuracy: 0.97419745
Accuracy: 0.97269726
Accuracy: 0.9720972
Accuracy: 0.96369636
Accuracy: 0.96369636
Train: 9
Accuracy: 0.11011101
Accuracy: 0.9108911
Accuracy: 0.930093
Accuracy: 0.94179416
Accuracy: 0.9420942
Accuracy: 0.9471947
Accuracy: 0.9558956
Accuracy: 0.95169514
Accuracy: 0.95739573
Accuracy: 0.95649564
Accuracy: 0.9582958
Accuracy: 0.95649564
Accuracy: 0.95559555
Accuracy: 0.96219623
Accuracy: 0.96159613
Accuracy: 0.9633963
Accuracy: 0.9576958
Accuracy: 0.9558956
Accuracy: 0.9633963
Accuracy: 0.9630963
Accuracy: 0.9660966
Accuracy: 0.9669967
Accuracy: 0.9630963
Accuracy: 0.9648965
Accuracy: 0.9690969
Accuracy: 0.97179717
Accuracy: 0.969697
Accuracy: 0.96459645
Accuracy: 0.9708971
Accuracy: 0.9705971
Accuracy: 0.9687969
Accuracy: 0.97359735
Accuracy: 0.969697
Accuracy: 0.96939695
Accuracy: 0.9678968
Accuracy: 0.9678968
Accuracy: 0.9720972
Accuracy: 0.9729973
Accuracy: 0.9720972
Accuracy: 0.9660966
Accuracy: 0.9702970